<h1 align='center'>Введение в обработку естественного языка</h1>  

<h2 align='center'>Урок 8. Рекуррентные нейронные сети RNN LSTM GRU</h2>  

<h3 align='left'>Практическое задание:</h3>  

Разобраться с моделькой перевода как она устроена (без механизма внимания), запустить для перевода с русского на английский (при желании можно взять другие пары языков)

In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

## Скачиваем и обрабатываем датасет

Мы будем использовать набор языковых данных, предоставленный http://www.manythings.org/anki/

In [2]:
!wget http://www.manythings.org/anki/rus-eng.zip

--2023-05-17 18:35:29--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15460248 (15M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.74M  6.38MB/s    in 2.3s    

2023-05-17 18:35:32 (6.38 MB/s) - ‘rus-eng.zip’ saved [15460248/15460248]



In [3]:
!mkdir rus-eng
!unzip rus-eng.zip -d rus-eng/

Archive:  rus-eng.zip
  inflating: rus-eng/rus.txt         
  inflating: rus-eng/_about.txt      


In [4]:
!ls /content/rus-eng/ -lah

total 74M
drwxr-xr-x 2 root root 4.0K May 17 18:35 .
drwxr-xr-x 1 root root 4.0K May 17 18:35 ..
-rw-r--r-- 1 root root 1.5K Apr  2 03:16 _about.txt
-rw-r--r-- 1 root root  74M Apr  2 03:16 rus.txt


In [5]:
# Путь к файлу
path_to_file = "/content/rus-eng/rus.txt"

Функция препроцессинга

In [6]:
def preprocess_sentence(w):
  w = w.lower().strip()

  # вставим пробелы между словом и следующими за ним знаками препинания
  # например: "he is a boy." => "he is a boy ."
  # Источник:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # заменим всё пробелами, кроме (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # добавление начального и конечного токенов в предложение
  # так модель будет знать, когда начать и закончить предсказание  
  w = '<start> ' + w + ' <end>'
  return w

Пример:

In [7]:
preprocess_sentence("I can't go.")

"<start> i can't go . <end>"

In [8]:
# 1. Уберём акценты
# 2. Вычислим предложения
# 3. Вернём пары слов в формате: [ENG, RUS]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [9]:
en, ru = create_dataset(path_to_file, None)
print(en[0])
print(ru[0])

<start> go . <end>
<start> марш ! <end>


In [10]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [11]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### Limit the size of the dataset to experiment faster (optional)


In [12]:
len(en), len(ru)

(467119, 467119)

In [13]:
# Попробуем поэкспериментировать с размером набора данных
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Подсчитаем максимальную длину (max_length) искомого тензора
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [14]:
# Создание обучающих и проверочных наборов с использованием разделения 80 на 20
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Посмотрим на их длину
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


In [15]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [16]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
19 ----> он
38 ----> е
115 ----> любит
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
29 ----> he
264 ----> loves
117 ----> her
3 ----> .
2 ----> <end>


### Создадим tf.data dataset

In [17]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [18]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 15]), TensorShape([64, 11]))

In [19]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=False,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [20]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# выборочный ввод
sample_hidden = encoder.initialize_hidden_state()
sample_hidden = encoder(example_input_batch, sample_hidden)
# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder Hidden state shape: (batch size, units) (64, 1024)


In [21]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

  def call(self, x, hidden):
    # enc_output shape == (batch_size, max_length, hidden_size)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

    # передача объединенного вектора в GRU
    output, state = self.gru(x, initial_state=hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [22]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)



In [23]:
decoder_sample_x.shape

TensorShape([64, 7362])

In [24]:
decoder_sample_h.shape

TensorShape([64, 1024])

## Определим оптимизатор и функцию потерь

In [25]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

Промежуточное сохранение результатов

In [26]:
checkpoint_dir = './training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [27]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - ввод таргета в качестве следующего входного сигнала
    for t in range(1, targ.shape[1]):
      # проброс enc_output в decoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden)

      loss += loss_function(targ[:, t], predictions)

      # использование teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [28]:
EPOCHS = 15

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # сохраняем (checkpoint) модель каждые 2 эпохи
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.7179
Epoch 1 Batch 100 Loss 2.0827
Epoch 1 Batch 200 Loss 1.7246
Epoch 1 Batch 300 Loss 1.6945
Epoch 1 Batch 400 Loss 1.5131
Epoch 1 Batch 500 Loss 1.4816
Epoch 1 Batch 600 Loss 1.3846
Epoch 1 Batch 700 Loss 1.4122
Epoch 1 Batch 800 Loss 1.1806
Epoch 1 Batch 900 Loss 1.1584
Epoch 1 Batch 1000 Loss 1.1129
Epoch 1 Batch 1100 Loss 1.1264
Epoch 1 Batch 1200 Loss 1.1389
Epoch 1 Loss 1.4754
Time taken for 1 epoch 85.10304570198059 sec

Epoch 2 Batch 0 Loss 0.9189
Epoch 2 Batch 100 Loss 0.9751
Epoch 2 Batch 200 Loss 0.9108
Epoch 2 Batch 300 Loss 0.9498
Epoch 2 Batch 400 Loss 0.9561
Epoch 2 Batch 500 Loss 0.8612
Epoch 2 Batch 600 Loss 0.8383
Epoch 2 Batch 700 Loss 0.8623
Epoch 2 Batch 800 Loss 0.8502
Epoch 2 Batch 900 Loss 0.8060
Epoch 2 Batch 1000 Loss 0.7457
Epoch 2 Batch 1100 Loss 0.6537
Epoch 2 Batch 1200 Loss 0.7273
Epoch 2 Loss 0.8253
Time taken for 1 epoch 55.40283751487732 sec

Epoch 3 Batch 0 Loss 0.5143
Epoch 3 Batch 100 Loss 0.5870
Epoch 3 Batch 200 Loss 0.502

## Перевод

* Функция оценки аналогична циклу обучения, за исключением того, что здесь мы не используем "teacher forcing". Входными данными для декодера на каждом временном шаге являются его предыдущие предсказания вместе со скрытым состоянием и выходными данными кодера.
* Прекратить прогнозирование, когда модель предскажет *конечный токен*.
* И сохранить значения *веса внимания для каждого временного шага*.

Примечание: Выходной сигнал энкодера вычисляется только один раз для одного входа.

In [29]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # сохранение весов внимания для последующего построения графика
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence

    # прогнозируемый идентификатор вводится обратно в модель
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [30]:
def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

## Восстановим последнюю контрольную точку и протестируем

In [31]:
# восстановим последнюю контрольную точку из checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [32]:
translate('Здесь хорошо.')

Input: <start> здесь хорошо . <end>
Predicted translation: it's well here . <end> 


In [33]:
translate('Я не смогу поехать.')

Input: <start> я не смогу поехать . <end>
Predicted translation: i can't go . <end> 


In [34]:
translate(u'Вы еще дома?')

Input: <start> вы еще дома ? <end>
Predicted translation: are you still home ? <end> 


In [35]:
translate(u'Вы все еще дома?')

Input: <start> вы все еще дома ? <end>
Predicted translation: are you still home ? <end> 


In [36]:
translate(u'Попробуй сделать это.')

Input: <start> попробуй сделать это . <end>
Predicted translation: try to do that . <end> 


In [37]:
translate(u'Я люблю, когда идет снег.')

Input: <start> я люблю , когда идет снег . <end>
Predicted translation: i like when it rains . <end> 


In [38]:
translate(u'Я никогда такого не делаю.')

Input: <start> я никогда такого не делаю . <end>
Predicted translation: i never do that . <end> 


**Вывод:**  

Видно, что короткие предложения переводятся достаточно хорошо. А вот если предложение досаточно длинное, то перевод становится заметно хуже.